In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
#KFold
#https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
#http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.metrics import mean_squared_error, make_scorer
#http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
from sklearn.ensemble import RandomForestRegressor

plt.style.use('ggplot') # Look Pretty

In [2]:
X = pd.read_csv('train_values.csv', index_col='row_id')
y = pd.read_csv('train_labels.csv', index_col='row_id')
df_predict = pd.read_csv('test_values.csv', index_col='row_id')

### Preview Dataset

In [ ]:
X.head()

In [ ]:
X.describe()

In [ ]:
X.shape

In [ ]:
X.isnull().sum()

In [ ]:
X.dtypes

### Unique Count

In [ ]:
#Check unique value for each column
def unique_counts(df):
    for i in df.columns:
        count = df[i].nunique()
        print(i, ": ", count)
        
unique_counts(X)

### Create Correction list and categorical list

In [3]:
def create_list(df, y):
    corr_list = []
    categorical_list = []
    numeric_count = 0
    
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.number):
            corr = (col, y[y.columns[0]].corr(X[col]))
            #print(col, ": ", corr)
            corr_list.append(corr)
        
            numeric_count += 1
            min_val = df[col].min()
            max_val = df[col].max()
            print(numeric_count, col, '\n Min:', min_val,'Max:', max_val, '\n')
        
        else:
            categorical_list.append(col)
            
    return corr_list, categorical_list

In [4]:
corr_list, X_cat_list = create_list(X, y)

1 econ__pct_civilian_labor 
 Min: 0.207 Max: 1.0 

2 econ__pct_unemployment 
 Min: 0.01 Max: 0.248 

3 econ__pct_uninsured_adults 
 Min: 0.046 Max: 0.496 

4 econ__pct_uninsured_children 
 Min: 0.012 Max: 0.281 

5 demo__pct_female 
 Min: 0.278 Max: 0.573 

6 demo__pct_below_18_years_of_age 
 Min: 0.092 Max: 0.41700000000000004 

7 demo__pct_aged_65_years_and_older 
 Min: 0.045 Max: 0.34600000000000003 

8 demo__pct_hispanic 
 Min: 0.0 Max: 0.932 

9 demo__pct_non_hispanic_african_american 
 Min: 0.0 Max: 0.858 

10 demo__pct_non_hispanic_white 
 Min: 0.053 Max: 0.99 

11 demo__pct_american_indian_or_alaskan_native 
 Min: 0.0 Max: 0.8590000000000001 

12 demo__pct_asian 
 Min: 0.0 Max: 0.341 

13 demo__pct_adults_less_than_a_high_school_diploma 
 Min: 0.01507537688442211 Max: 0.4735264735264735 

14 demo__pct_adults_with_high_school_diploma 
 Min: 0.06532663316582914 Max: 0.5589123867069486 

15 demo__pct_adults_with_some_college 
 Min: 0.10954773869346733 Max: 0.47395301327885603 

16

### Preview categorical columns

In [ ]:
#print unique value for catergorical columns
for i in X_cat_list:
    value = X[i].unique()
    print(i, ": ", value,'\n')

In [ ]:
#graph plotting for catergorical columns
for col in X_cat_list:
    X[col].value_counts().plot(kind='bar')
    plt.title(col)
    plt.show()

### Checking corelation of catergorical columns and y label

In [5]:
#select feature with significant correlation with rate of heart disease
corr_rate = 0.5
corr_plot_list = []
for feature in corr_list:
    if abs(feature[1]) > corr_rate:
        print(feature)
        corr_plot_list.append(feature[0])
#sorted(corr_list, key=lambda x: x[1])

('demo__pct_adults_less_than_a_high_school_diploma', 0.5273816184439839)
('demo__pct_adults_bachelors_or_higher', -0.5413849922585271)
('health__pct_adult_obesity', 0.5937750422550884)
('health__pct_diabetes', 0.631765016185405)
('health__pct_physical_inacticity', 0.6503046257575985)


In [ ]:
#create df for scatter plot
df_scatter = pd.concat([X[corr_plot_list],y], axis=1)
df_scatter.head()

In [ ]:
df_scatter.columns = [
    'Does not have a high school diploma',
    'Bachelor\'s degree or higher',
    'Adults who obese',
    'Population with diabetes',
    'Adult that is physically inactive',
    'Rate of heart disease'
]
pd.plotting.scatter_matrix(df_scatter, alpha = 0.2, figsize = (18, 18), diagonal = 'kde')
plt.show()

In [ ]:
#take log to 'Rate of heart disease'
df_scatter['ln_Rate of heart disease'] = np.log(df_scatter['Rate of heart disease']+1)
pd.plotting.scatter_matrix(df_scatter.drop(['Rate of heart disease'], axis=1), alpha = 0.2, figsize = (18, 18), diagonal = 'kde')
plt.show()

In [ ]:
df_scatter.drop(['Rate of heart disease'], axis=1).corr()#.to_csv('corr.csv')

In [ ]:
#Box plot
def create_boxplot(X,y):
    df = pd.concat([X,y], axis=1)
    for col in X_cat_list:
        df.boxplot(column=y.columns[0], by=col, figsize = (20, 10))
        plt.show()

In [ ]:
create_boxplot(X,y)

### Histogram of y label

In [ ]:
y['heart_disease_mortality_per_100k'].plot.hist(alpha=0.5)
plt.title('Rate of heart disease (per 100,000 individuals)')
plt.xlabel('Rate')
plt.show()

### Check median values by area__rucc

In [8]:
df_X_metro = X.copy()
df_X_metro['Metro?'] = 'Nonmetro'
df_X_metro.loc[X['area__rucc'].str[:5] == 'Metro', 'Metro?']= 'Metro'

na_list = ['health__pct_adult_smoking', 
           'health__pct_low_birthweight',
           'health__pct_excessive_drinking', #
           'health__air_pollution_particulate_matter',
           'health__motor_vehicle_crash_deaths_per_100k', #median()
           'health__pop_per_dentist', #median()
           'health__pop_per_primary_care_physician' #median()
          ] 

#df_X_metro.groupby('Metro?')[na_list].mean()
df_X_metro.groupby('Metro?')[na_list].median()

,health__pct_adult_smoking,health__pct_low_birthweight,health__pct_excessive_drinking,health__air_pollution_particulate_matter,health__motor_vehicle_crash_deaths_per_100k,health__pop_per_dentist,health__pop_per_primary_care_physician
Metro?,,,,,,,
Metro,0.1980,0.080,0.165,12.0,14.06,2179.0,1769.5
Nonmetro,0.2165,0.081,0.162,12.0,22.84,2940.0,2104.5


In [ ]:
df_X_metro.head()

In [ ]:
df_X_metro.boxplot(column='health__homicides_per_100k', by='area__rucc', figsize = (20, 10))
plt.show()

In [10]:
X_cat_list

['area__rucc', 'area__urban_influence', 'econ__economic_typology', 'yr']

In [12]:
#median by categorical_list
X.groupby('econ__economic_typology')[na_list].median()

,health__pct_adult_smoking,health__pct_low_birthweight,health__pct_excessive_drinking,health__air_pollution_particulate_matter,health__motor_vehicle_crash_deaths_per_100k,health__pop_per_dentist,health__pop_per_primary_care_physician
econ__economic_typology,,,,,,,
Farm-dependent,0.1830,0.073,0.1935,11.0,26.305,3089.0,2289.0
Federal/State government-dependent,0.2030,0.083,0.1595,12.0,17.300,2480.0,1914.0
Manufacturing-dependent,0.2190,0.080,0.1540,13.0,20.580,3270.0,2219.5
Mining-dependent,0.2235,0.085,0.1530,10.0,26.290,3215.0,2319.0
Nonspecialized,0.2130,0.083,0.1560,12.0,18.040,2519.0,1879.0
Recreation,0.1990,0.072,0.1790,11.0,16.830,2255.0,1619.0


### Remove columns/ rows and Replace NaN values

In [ ]:
X.drop(['health__homicides_per_100k'], axis=1, inplace=True)
#Require at least 20/33 non-NA values
#X.dropna(thresh=20, inplace=True)
#fill na with mean
#X.fillna(X.median(), inplace=True)
#fill na with group median
X.fillna(X.groupby(X_cat_list).transform('median'), inplace=True)
X['health__air_pollution_particulate_matter']=X['health__air_pollution_particulate_matter'].astype(str)
X.head()

In [ ]:
df_predict.drop(['health__homicides_per_100k'], axis=1, inplace=True)

#fill na with group median
df_predict.fillna(df_predict.groupby(X_cat_list).transform('median'), inplace=True)
df_predict['health__air_pollution_particulate_matter']=df_predict['health__air_pollution_particulate_matter'].astype(str)

### Take log and check min, max for numeric columns, put catergorical column name to cat_list

In [ ]:
def data_transform(df):
    count = 0
    for col in df.columns:       
        if np.issubdtype(df[col].dtype, np.number):
            count += 1
            print(count,'.Updating column:',col)
            df[col] = np.log(df[col]+1)
    return df

In [ ]:
X = data_transform(X)
df_predict = data_transform(df_predict)

In [ ]:
X.head()

### Graph Plotting

In [ ]:
def plot_ind(df, indicator, diagonal_type):
    df_filtered = df.filter(regex=indicator)
    pd.plotting.scatter_matrix(df_filtered, alpha = 0.2, figsize = (20, 20), diagonal = diagonal_type)
    return plt.show()

In [ ]:
plot_ind(X, 'econ__pct','kde')

In [ ]:
plot_ind(X, 'demo__','kde')

In [ ]:
plot_ind(X, 'health__','kde')

### One Hot, Train_Test Split and Feature Scaling

In [ ]:
def one_hot(df, cat_list):
    return pd.get_dummies(df, columns = cat_list) #get_dummies as Onehot

In [ ]:
#get_dummies as Onehot
X = one_hot(X, X_cat_list)
df_predict = one_hot(df_predict, X_cat_list)

In [ ]:
#train:Test = 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), train_size =0.80, test_size = 0.2, random_state=154)

In [ ]:
#feature scaling
std_scale = StandardScaler().fit(X_train)

X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

X_predict_std = std_scale.transform(df_predict)

#set cv by K-Fold
kf = KFold(n_splits=30, shuffle=True, random_state=52)

### Baseline Model

In [ ]:
##########Baseline case only##########
print('##########Baseline Model##########')
lr = linear_model.LinearRegression()
lr.fit(X_train_std , y_train)

lr_train_scores = cross_val_score(lr, X_train_std, y_train, scoring='neg_mean_squared_error', cv=kf)
lr_test_scores = cross_val_score(lr, X_test_std, y_test, scoring='neg_mean_squared_error', cv=kf)

print('RMSE for Train set: %.2f' % abs(lr_train_scores.mean())**(1/2))
print('RMSE for Test set: %.2f' % abs(lr_test_scores.mean())**(1/2))

In [ ]:
#Finding best parameter for model
def find_best_params(model_name, grid_values):
    model = model_name
    grid_search = GridSearchCV(estimator = model, param_grid = grid_values, scoring='neg_mean_squared_error')
    # Fit the grid search to the data
    grid_search.fit(X_train_std, y_train)
    
    grid_search_scores = cross_val_score(grid_search, X_train_std, y_train, cv=kf, scoring='neg_mean_squared_error')

    print('Grid best parameter: ', grid_search.best_params_)
    print('Best score: %.2f' % abs(grid_search.best_score_)**(1/2))
    print("Accuracy: %0.2f (+/- %0.2f)" % (abs(grid_search_scores.mean())**(1/2), grid_search_scores.std() * 2))

### Ridge Regression

In [ ]:
grid_values = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 25, 30, 50]}
model_name = linear_model.Ridge()

find_best_params(model_name, grid_values)

In [ ]:
print('##########Ridge Regression Test Set Result##########')

ridge_lr = linear_model.Ridge(alpha=10)
ridge_lr_train_scores = cross_val_score(ridge_lr, X_train_std, y_train, scoring='neg_mean_squared_error', cv=kf)
ridge_lr_test_scores = cross_val_score(ridge_lr, X_test_std, y_test, scoring='neg_mean_squared_error', cv=kf)

print('RMSE for Train set: %.2f' % abs(ridge_lr_train_scores.mean())**(1/2))
print('RMSE for Test set: %.2f' % abs(ridge_lr_test_scores.mean())**(1/2))

### Lasso Regression

In [ ]:
"""
print('##########Lasso##########')
lassocv_lr = linear_model.LassoCV()
lassocv_lr.fit(X_train_std , y_train)

#y_cv_lassocv_lr = lassocv_lr.predict(X_cv_std)
#print('RMSE for CV set: %.2f' %(mean_squared_error(y_cv, y_cv_lassocv_lr))** (1/2))

#########After best parameter is selected#########
#y_test_lassocv_lr = lassocv_lr.predict(X_test_std)
#print('RMSE for test set: %.2f' %(mean_squared_error(y_test, y_test_lassocv_lr))** (1/2))

lassocv_lr_scores = cross_val_score(lassocv_lr, X_train, y_train, cv=30, scoring='neg_mean_squared_error')

print('RMSE for Train set: %.2f' % abs(lassocv_lr_scores.mean())**(1/2))
print('RMSE for Test set:')"""

In [ ]:
grid_values = {'alpha': [0.01, 0.02, 0.03, 0.05, 0.1, 1, 10], 'max_iter':[10000,5000]}

model_name = linear_model.Lasso()
find_best_params(model_name, grid_values)

In [ ]:
print('##########Lasso Regression Test Set Result##########')

lasso_lr = linear_model.Lasso(alpha=0.05, max_iter=10000)
lasso_lr_train_scores = cross_val_score(lasso_lr, X_train_std, y_train, scoring='neg_mean_squared_error', cv=kf)
lasso_lr_test_scores = cross_val_score(lasso_lr, X_test_std, y_test, scoring='neg_mean_squared_error', cv=kf)

print('RMSE for Train set: %.2f' % abs(lasso_lr_train_scores.mean())**(1/2))
print('RMSE for Test set: %.2f' % abs(lasso_lr_test_scores.mean())**(1/2))

### ElasticNet

In [ ]:
grid_values = {'alpha': [0.01, 0.02, 0.03, 0.05, 0.1, 1, 10], 'l1_ratio': [0.1, 0.5, 0.7, 0.9, 1]}

model_name = linear_model.ElasticNet()
find_best_params(model_name, grid_values)

In [ ]:
print('##########ElasticNet Test Set Result##########')

en = linear_model.ElasticNet(alpha=0.02, l1_ratio=0.9)
en_train_scores = cross_val_score(en, X_train_std, y_train, scoring='neg_mean_squared_error', cv=kf)
en_test_scores = cross_val_score(en, X_test_std, y_test, scoring='neg_mean_squared_error', cv=kf)

print('RMSE for Train set: %.2f' % abs(en_train_scores.mean())**(1/2))
print('RMSE for Test set: %.2f' % abs(en_test_scores.mean())**(1/2))

### Prediction

In [ ]:
def prediction(model):
    
    #create dataframe for answer
    df = pd.DataFrame(df_predict.index)

    model.fit(X_train_std, y_train)

    #return the result of predict
    df['heart_disease_mortality_per_100k'] = pd.Series(model.predict(X_predict_std))
    df['heart_disease_mortality_per_100k'] =df['heart_disease_mortality_per_100k'].astype('int')
    
    return df

In [ ]:
#df = prediction(linear_model.Ridge(alpha=10)) #32.1999
#df = prediction(linear_model.Lasso(alpha=0.05)) #32.1632 #32.1086 by group median
#df = prediction(linear_model.LinearRegression()) #32.2258
#df = prediction(RandomForestRegressor(bootstrap=True, max_depth=15, max_features=20, min_samples_leaf=3, min_samples_split=3)) #35.7147

df.to_csv('test_labels.csv', index=False)

### Random Forest

In [ ]:
grid_values = {
    'bootstrap': [True],
    'max_depth': [15, 20, 25],
    'max_features': [10, 15, 20],
    'min_samples_leaf': [3, 5, 8],
    'min_samples_split': [3, 5, 8],
}
model_name = RandomForestRegressor()
find_best_params(model_name, grid_values)

In [ ]:
#n_estimators, max_features, max_depth, min_samples_split 
#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
print('##########Random Forest Test Set Result##########')

rf = RandomForestRegressor(bootstrap=True, max_depth=15, max_features=20, min_samples_leaf=3, min_samples_split=3)
rf_train_scores = cross_val_score(rf, X_train_std, y_train, scoring='neg_mean_squared_error', cv=kf)
rf_test_scores = cross_val_score(rf, X_test_std, y_test, scoring='neg_mean_squared_error', cv=kf)

print('RMSE for Train set: %.2f' % abs(rf_train_scores.mean())**(1/2))
print('RMSE for Test set: %.2f' % abs(rf_test_scores.mean())**(1/2))

### Feature Importances

In [ ]:
rf.fit(X_train_std, y_train)
df_feature = pd.DataFrame(X.columns, columns=['feature'])

#return the result of predict
df_feature['feature_importances'] = pd.Series(rf.feature_importances_)

In [ ]:
#feature_importances of catergorical features
cat_list = ['area__rucc', 'area__urban_influence', 'econ__economic_typology', 'health__air_pollution_particulate_matter', 'yr']
df_feature[df_feature['feature'].isin(cat_list)]

In [ ]:
#Top 10 features
df_feature.sort_values(by=['feature_importances'], ascending=False).head(10)

### Others

In [ ]:
#https://stackoverflow.com/questions/42228735/scikit-learn-gridsearchcv-with-multiple-repetitions/42230764#42230764
#https://stackoverflow.com/questions/42362027/model-help-using-scikit-learn-when-using-gridsearch/42364900#42364900
kf = KFold(n_splits=5, shuffle=True, random_state=52)
gcv = GridSearchCV(pipe, param_grid = grid_values, cv=cv)

gcv.fit(features,labels) #with the full dataset

for train_ind, test_ind in cv.split(features,labels):
    x_train, x_test = features[train_ind], features[test_ind]
    y_train, y_test = labels[train_ind],labels[test_ind]

    gcv.best_estimator_.fit(x_train,y_train)
    gcv.best_estimator_.predict(x_test)
#################################################################    
for train_index, test_index in kf.split(X):
...    print("TRAIN:", train_index, "TEST:", test_index)
...    X_train, X_test = X[train_index], X[test_index]
...    y_train, y_test = y[train_index], y[test_index]
TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1] TEST: [2 3]

In [ ]:
kf = KFold(n_splits=30, shuffle=True, random_state=52)
for train_index, test_index in kf.split(X,y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
##############Test#################
#std_scale = StandardScaler().fit(X)

grid_values = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]}

cv = KFold(n_splits=10, shuffle=True, random_state=52)
gcv = GridSearchCV(ridge_lr, param_grid = grid_values, cv=cv)

#gcv.fit(features,labels) #with the full dataset
gcv.fit(X, y)

for train_ind, test_ind in cv.split(X, y):
    x_train, x_test = X[train_ind], X[test_ind]
    y_train, y_test = y[train_ind],y[test_ind]

    gcv.best_estimator_.fit(x_train,y_train)
    gcv.best_estimator_.predict(x_test)